In [1]:
import DualNetM as dm
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["figure.figsize"] = [6, 4.5]

In [26]:
prior_network = dm.datasets.load_mouse_prior_network()
adata = sc.read_csv ('/home/dbj/DeepRIG-main/Datasets/500_ChIP-seq_mHSC-GM/500_ChIP-seq_mHSC-GM-ExpressionData.csv')
adata=adata.T

In [27]:
adata.var_names = adata.var_names.str.capitalize()

In [28]:
data = dm.data_preparation(adata, prior_network)

Data loading and preprocessing....
    15573 extra edges (Spearman correlation > 0) are added into the prior network.
    Total number of edges: 32248.
 n_genes x n_cells =600 x 889


In [29]:
DualNetM_GRN_model = dm.NetModel(epochs=340, cuda='0', seed=9)
DualNetM_GRN_model.run(data)

  Iter: 1/1: 100%|███████████████| 340/340 [00:10<00:00, 33.23it/s, loss=0.00107, min_loss=0.000106]


In [33]:
DualNetM_GRN_model.get_network('/home/dbj/DeepRIG-main/Datasets/500_ChIP-seq_mHSC-GM')

In [44]:
GRN=pd.read_csv('/home/dbj/DeepRIG-main/Datasets/500_ChIP-seq_mHSC-GM/outputGRN.csv',header=0)
GRN.columns = ['Gene1', 'Gene2', 'weights_combined']

In [45]:
GRN

,Gene1,Gene2,weights_combined
0,Vdr,Ap3s1,1.255033
1,Vdr,Stxbp4,4.340764
2,Vdr,Ndrg1,4.733782
3,Vdr,Tbxas1,1.118022
4,Vdr,Smarca2,3.126126
...,...,...,...
32167,Ms4a6c,Ebf1,0.360613
32168,Ms4a6c,Tbxas1,0.246428
32169,Ms4a6c,Gria3,1.066971
32170,Ms4a6c,Ms4a3,0.371934


In [49]:
True_GRN=pd.read_csv('/home/dbj/DeepRIG-main/Datasets/500_ChIP-seq_mHSC-GM/500_ChIP-seq_mHSC-GM-network.csv')
True_GRN['Gene1']=True_GRN['Gene1'].str.capitalize()
True_GRN['Gene2']=True_GRN['Gene2'].str.capitalize()

In [50]:
True_GRN

,Gene1,Gene2,Score
0,Cbfb,Msh6,2386.0
1,Cbfb,Slc2a3,1502.0
2,Cbfb,Nfe2,2138.0
3,Cbfb,Arid5a,1700.0
4,Cbfb,Tmem229b,1785.0
...,...,...,...
7359,Wdr5,Pstpip1,113.0
7360,Wdr5,Rps6ka3,110.0
7361,Wdr5,Adgrg3,108.0
7362,Wdr5,Sell,105.0


In [52]:
TFs = set(True_GRN['Gene1'])
Genes = set(True_GRN['Gene1'])| set(True_GRN['Gene2'])

In [54]:
EPR=dm.eval_utils.EarlyPrec(trueEdgesDF=True_GRN,predEdgesDF=GRN)
EPR

Calculating the EPR(early prediction rate)...
  Consider only edges going out of source genes
  7364 TF Edges in ground-truth
  5720 Predicted TF edges are considered
  5720 Top-k edges selected
  4069 true-positive edges


1.0185162797065217

In [56]:
AUPRC,AUPRC_ratio=dm.eval_utils.evaluateAUPRratio(output=GRN,label=True_GRN,TFs=TFs,Genes=Genes)
AUPRC,AUPRC_ratio

(0.6904416318374993, 1.274748020975372)

In [58]:
AUROC=dm.eval_utils.evaluateAUROC(output=GRN,label=True_GRN)
AUROC

0.8921379396878073